<a href="https://colab.research.google.com/github/dangcaptkd/CS114.K21-/blob/master/sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sarcasm detection**

**Data**
  *   Data train được lấy từ kaggle: News Headlines Dataset For Sarcasm Detection
  *   Dataset-train gồm:
    *   is_sarcastic: là 1 nếu là sarcasm ngược lại là 0
    *   headline: tiêu đề
    *   article_link: link của article


**Các kĩ thuật xử lý data**

  * Regular Expression: loại bỏ các kí tự không cần thiết. Ex: don't ---> don t
  * Stemming data: đưa các từ về hình thức gốc của nó. Ex: loved---> love
  * TF-IDF (Term frequency-Document term frequency): 
    * TF: là số lần xuất hiện của từ đó trong văn bản
    * IDF: Tần số nghịch của 1 từ trong tập văn bản 
    * tfidf = TF*IDF, những từ có tần số ifidf cao là những từ xuất hiện nhiều trong văn bản này, và xuất hiện ít trong các văn bản khác. Việc này giúp lọc ra những từ phổ biến và giữ lại những từ có giá trị cao (từ khoá của văn bản đó).










Lấy data từ kaggle!



In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"quyennguyenthinh","key":"52c200c7cee98c6a1dd7c6433c61afd8"}'}

In [2]:
#!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 111MB/s]


In [4]:
!unzip news-headlines-dataset-for-sarcasm-detection

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


# Load,show, check, pre-processing, training and testing data
#(regular expression, steming, TF-IDF Vectorizer)

In [5]:
import pandas as pd
import numpy as np
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

load data from jason file

In [6]:
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines = True)

show data

In [7]:
data

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall,0
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes,0
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0


Check the null value

In [8]:
print(data.isnull().any(axis = 0))

article_link    False
headline        False
is_sarcastic    False
dtype: bool


**pre-processing data**

Ở cột headline sẽ có 1 vài kí tự cần được loại bỏ('?.+-="/...), dùng regular expression để loại bỏ các kí tự này

In [9]:
print('Example')
print('before:',data['headline'][0])
data['headline'] = data['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))
print('after: {}'.format(data['headline'][0]))

Example
before: former versace store clerk sues over secret 'black code' for minority shoppers
after: former versace store clerk sues over secret  black code  for minority shoppers


Gán feature và label 

In [10]:
features = data['headline']
labels = data['is_sarcastic']

**Stemming**

Stemming là 1 kĩ thuật đưa các từ về hình thức gốc của nó, ex: loved-->love

In [11]:
print('Example')
print('before steaming:', features[0])
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))
print('after steaming:', features[0])

Example
before steaming: former versace store clerk sues over secret  black code  for minority shoppers
after steaming: former versac store clerk sue over secret black code for minor shopper


**TF-IDF Vectorizer**

*   TfidfVectorizer(): "Convert a collection of raw documents to a matrix of TF-IDF features."
*   TfidfVectorizer.fit_transform(): "Learn vocabulary and idf, return **document-term matrix**"

Document-term matrix là ma trận có các hàng tương ứng với văn bản, các cột tương ứng với từng từ

Ex:
 
![alt text](https://www.researchgate.net/profile/Khawar_Mehmood/publication/342027900/figure/tbl2/AS:900374928031745@1591677519266/Example-of-document-term-matrix-Unigram.png)






In [12]:
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
features = tv.fit_transform(features).toarray()

Import thư viện cần thiết

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

split và show shape data

In [14]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.1, random_state = 0)
print('features_train:', features_train.shape)
print('labels_train:', labels_train.shape)
print('features_test', features_test.shape)
print('labels_test:', labels_test.shape)

features_train: (24038, 5000)
labels_train: (24038,)
features_test (2671, 5000)
labels_test: (2671,)


**training and testing data**

Sử dụng các model để train: LinearSVC, GausinanNB, LogisticRegression, DecisionTreeClassifier, RandomForestClassifier

In [15]:
models = []
models.append(('LinearSVC', LinearSVC()))
models.append(('GaussianNB', GaussianNB()))
models.append(('LogisticRegression', LogisticRegression()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('RandomForestClassifier', RandomForestClassifier()))

In [17]:
for name, model in models:
  model.fit(features_train, labels_train)
  print(name)
  print('\tscore train:', model.score(features_train, labels_train))
  print('\tscore test:', model.score(features_test, labels_test))

LinearSVC
	score train: 0.9110574923038522
	score test: 0.8360164732309996
GaussianNB
	score train: 0.7826358266078709
	score test: 0.7162111568700861
LogisticRegression
	score train: 0.8824777435726766
	score test: 0.8375140396855111
DecisionTreeClassifier
	score train: 0.999875197603794
	score test: 0.7487832272557094
RandomForestClassifier
	score train: 0.999875197603794
	score test: 0.8124298015724448


# Crawl data

In [18]:
import requests
from bs4 import BeautifulSoup
import numpy as np

In [70]:
def crawl(url, data):
  r1 = requests.get(url)
  coverpage = r1.content
  soup1 = BeautifulSoup(coverpage, 'html5lib')
  coverpage_news = soup1.find_all("h2", class_="sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE")
  for i in coverpage_news:
    data.append(i.get_text())
  

In [103]:
url = "https://www.theonion.com/latest"
more_stories = ''
data= []
while len(data)<2000:
  crawl(url+more_stories, data)
  r1 = requests.get(url)
  coverpage = r1.content
  soup1 = BeautifulSoup(coverpage, 'html5lib')
  t = soup1.find_all("a", class_="sc-1out364-0 hMndXN js_link")
  more_stories = t[-1].get('href')


In [104]:
len(data)

2000

In [105]:
data

['Mental Health Experts Advise On Best Ways To Combat Intrusive Thoughts Of Your Father Naked',
 'The Onion’s Father’s Day Gift Guide For The Dad Who Has It All',
 'Staples Center Employee Realizes He Left Shot Clock Buzzer On This Whole Time',
 'Father’s Day Gifts That Will Make Your Dad Feel Exactly The Way He Already Feels',
 'Huh, Boyfriend’s Ex Just Made Interesting Hair Choice',
 'Mom Comes Back From Long Call With Friend Looking Real Sad',
 '‘It’s Perfect Outside,’ Announces Sweating Woman Slowly Losing Consciousness In Middle Of Heatstroke',
 'Man Hates How Hot Dog-Eating Contests Reduce Art Of Eating Hot Dogs To A Competition',
 'Kristen Stewart To Play Princess Diana In Upcoming Biopic',
 'Resigned Pew Research Study Has No Fucking Clue What’s Going On With 15% Of Americans',
 'Aunt Jemima, Mrs. Butterworth, Uncle Ben’s Brands To Be Overhauled',
 'Traveling Contortionist Excited To Get Off The Road And Finally Stop Living Inside Suitcase',
 'White House Toilet Doesn’t Know If

In [107]:
is_sarcasm = [1]*2000
print(is_sarcasm)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [108]:
data = pd.DataFrame({'headline': data, 'is_sarcasm': is_sarcasm})
data

,headline,is_sarcasm
0,Mental Health Experts Advise On Best Ways To C...,1
1,The Onion’s Father’s Day Gift Guide For The Da...,1
2,Staples Center Employee Realizes He Left Shot ...,1
3,Father’s Day Gifts That Will Make Your Dad Fee...,1
4,"Huh, Boyfriend’s Ex Just Made Interesting Hair...",1
...,...,...
1995,"Over 500,000 Americans Sign Petition Demanding...",1
1996,Jeff Bezos Depressed After Realizing Net Worth...,1
1997,"Birch Tree Pictures Nude 4,000-Year-Old Bristl...",1
1998,NRA Receives Massive Funding Increase From Don...,1
